In [35]:
# This is an example of how we see 
# the package work. The functions listed here
# are probably the only ones that should be exposed, ie documented.
# others should br prepended with a double underscore
#  
# The cognet directory has the following "modules"
# which are seprate .py files containing clases and functions
# The modules are cognet.py, dataFormatter.py, model.py, util.py, viz.py
# we will write the viz.py later.

## Hypothesis generator for the GSS Data:
from hypothesis import Hypothesis

import sys, os
from tqdm import tqdm
from quasinet.qnet import qdistance
from cognet.cognet import cognet as cg
from cognet.dataFormatter import dataFormatter
from cognet.model import model 
from IPython.display import Image
from IPython.core.display import HTML
import pandas as pd
import numpy as np

yr = '2018'
POLEFILE='examples_data/polar_vectors.csv'
QPATH='examples_data/gss_'+yr+'.joblib'
IMMUTABLE_FILE='examples_data/immutable.csv'
GSSDATA = 'examples_data/gss_'+yr+'.csv'

In [36]:
## read in data, configure to Qnet specifications and fit Qnet model
fit = False
data_obj=dataFormatter(GSSDATA)
model_obj = model()
if fit:
    model_obj.fit(data_obj=data_obj)
    model_obj.save("../cognet/qnets/qnetsGSS_cognet_test.joblib")
else:
    model_obj.load("../cognet/qnets/gss_2018.joblib")

In [37]:
cognet_obj = cg()


cognet_obj.load_from_model(model_obj, data_obj, 'all')
cognet_obj.dmat_filewriter("GSS_cognet.py", "../cognet/qnets/gss_2018.joblib",
                           MPI_SETUP_FILE="GSS_mpi_setup.sh",
                           MPI_RUN_FILE="GSS_mpi_run.sh",
                           MPI_LAUNCHER_FILE="GSS_mpi_launcher.sh",
                           YEARS='2018',NODES=4,T=14)
cognet_obj.samples


,wrkstat,HRS1,HRS2,evwork,wrkslf,wrkgovt,OCC10,PRESTG10,PRESTG105PLUS,INDUS10,...,neisafe,rlooks,rgroomed,rweight,rhlthend,wtss,wtssnr,wtssall,vstrat,vpsu
0,temp not working,e,c,nan,someone else,private,b,c,c,c,...,very safe,nan,nan,nan,nan,e,e,e,3301,1
1,working fulltime,c,e,nan,someone else,private,b,d,d,c,...,very safe,nan,nan,nan,nan,c,c,c,3301,1
2,working fulltime,c,e,nan,someone else,private,c,d,d,c,...,very safe,attractive,about average,about the right weight,nan,c,c,c,3301,1
3,working fulltime,e,e,nan,someone else,private,c,c,c,c,...,somewhat safe,about average,about average,slightly underweight,excellent,c,c,c,3301,1
4,working fulltime,c,e,nan,self-employed,private,c,c,c,c,...,very unsafe,very unattractive,very poorly groomed,very overweight,poor,c,c,c,3301,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1779,retired,e,e,yes,someone else,private,d,c,c,c,...,very safe,nan,nan,nan,nan,c,c,c,3378,2
1780,school,e,e,no,nan,nan,e,e,e,e,...,somewhat safe,about average,about average,slightly underweight,excellent,d,d,d,3378,2
1781,working fulltime,c,e,nan,someone else,government,c,c,c,c,...,very safe,nan,nan,nan,nan,c,c,c,3378,2
1782,working parttime,c,e,nan,someone else,private,c,b,b,c,...,very safe,about average,about average,slightly overweight,excellent,c,c,c,3378,2


In [38]:
## Generating a Tree File

outdirname = 'dotfiles_GSS'
if not os.path.isdir(outdirname):
    os.makedirs(outdirname)
for f in os.listdir(outdirname):
    os.remove(os.path.join(outdirname, f))

outfilename = outdirname + "/GSS_tree.png"
model_obj.export_dot(outfilename, generate_trees = False)

! cd 'dotfiles_GSS'; for i in `ls *dot`; do dot -Tpng $i -o ${i/dot/png}; done >& /dev/null

In [39]:
! ls -lhS dotfiles_GSS/*png | head -n 3

## Viewing the generated tree file
Image(filename=outfilename)

-rw-rw-r-- 1 lrmantovani pi-ishanu 80K Nov 13 15:47 dotfiles_GSS/GSS_tree.png


In [41]:
## Generating Hypothesis

hypothesis = Hypothesis(model_path = outdirname, detailed_labels = True,
                       no_self_loops = False, causal_constraint = -2)

hypothesis.to_dot('hypothesis_GSS.dot')
hypothesis.hypothesis.sort_values('src')


Exception: qnet_orchestrator must be provided to Hypothesis